In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv("ml-25m/movies.csv") 
ratings = pd.read_csv("ml-25m/ratings.csv") 

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
#Filter data to include movies with at least 14500 ratings and users with at least 250 ratings

data = ratings.groupby('movieId').filter(lambda x: len(x) >= 14500)
data = data.groupby('userId').filter(lambda x: len(x) >= 250)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407450 entries, 51820 to 24996197
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     407450 non-null  int64  
 1   movieId    407450 non-null  int64  
 2   rating     407450 non-null  float64
 3   timestamp  407450 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 15.5 MB


In [3]:
#Create summaries of the dataset containing the ratings (i.e. counts and means)

stats = data.groupby('movieId')[['rating']].describe()

In [4]:
# stats.info()
# stats.head()

In [5]:
#Create the weighted average for each movie using the IMDB weighted average rating formula

V = stats['rating','count']
R = stats['rating','mean']
C = stats['rating','mean'].mean()
m = float(10000)

stats['rating','weighted_average'] = ((R*V) + (C*m)) / (V+m)

In [6]:
# stats.info()
# stats.head()

In [7]:
#Merge the weighted averages with the movies dataset 

movies_ratings_weighted = pd.merge(left=movies, right=stats, left_on='movieId', right_on='movieId')

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [8]:
#Sort new dataset in descending order according the the weighted average ratings

movies_ratings_weighted = movies_ratings_weighted.sort_values(('rating','weighted_average'), ascending=False)

In [13]:
#Shows the movies recommended in descending order like a recommender system would

movies_ratings_weighted[['title','genres',('rating','weighted_average')]]

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,title,genres,"(rating, weighted_average)"
42,"Shawshank Redemption, The (1994)",Crime|Drama,3.704187
38,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,3.703415
92,"Godfather, The (1972)",Crime|Drama,3.698744
205,"Matrix, The (1999)",Action|Sci-Fi|Thriller,3.691464
119,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,3.691412
...,...,...,...
28,Waterworld (1995),Action|Adventure|Sci-Fi,3.485159
243,Charlie's Angels (2000),Action|Comedy,3.481813
7,Ace Ventura: When Nature Calls (1995),Comedy,3.476548
26,Judge Dredd (1995),Action|Crime|Sci-Fi,3.471834


In [10]:
#Find the 25 movies with the highest weighted average ratings to recommend

movies_ratings_weighted_indices = movies_ratings_weighted['movieId'][0:25]

In [11]:
#Create a list of all unique users

userID_list = data['userId'].unique()

In [14]:
#Calculate average precision, recall, and F1 scores

precision_matrix = np.zeros(shape=(len(userID_list)))
recall_matrix = np.zeros(shape=(len(userID_list)))
f1_matrix = np.zeros(shape=(len(userID_list)))

for idx,i in enumerate(userID_list):
    userdf = data[data['userId']==i]
    top_movies_rated = userdf[userdf['rating']>=3.5]

    top_rated_indices = top_movies_rated['movieId'].values
    
    tp = len(np.intersect1d(movies_ratings_weighted_indices, top_rated_indices))
    fp = len(movies_ratings_weighted_indices)-tp
    fn = len(top_rated_indices)-tp

    precision = tp / (tp + fp + 1e-10)
    precision_matrix[idx] = precision
    
    recall = tp / (tp + fn + 1e-10)
    recall_matrix[idx] = recall

    f1 = 2.0 * (precision * recall) / (precision + recall + 1e-10)
    f1_matrix[idx] = f1


avg_precision = np.average(precision_matrix)
avg_recall = np.average(recall_matrix)
avg_f1 = np.average(f1_matrix)
print('Average Precision: ', avg_precision)
print('Average Recall: ', avg_recall)
print('Average F1 Score: ', avg_f1)

Average Precision:  0.8504183535728467
Average Recall:  0.11872195741399366
Average F1 Score:  0.20626861389033235
